In [1]:
import pandas as pd

In [2]:
df_receita = pd.read_csv('dados/receitas_candidatos_SP.csv', encoding='latin-1', sep=';')
df_receita = df_receita[df_receita['Situação de candidatura']=='Apto']
#df_receita = df_receita[['Nome candidato', 'Situação de totalização']].drop_duplicates()

In [40]:
df_votos = pd.read_csv('dados/votacao_candidato-uf_deputado_federal_2022_sp.csv', encoding='latin-1', sep=';')
df_votos = df_votos[['nm_candidato', 'sg_partido', 'ds_composicao_coligacao', 'ds_sit_totalizacao', 'pc_votos_validos', 'qt_votos_nom_validos', 'qt_votos_concorrentes']]

In [50]:
df_qe = pd.read_csv('dados/quociente-eleitoral.csv', encoding='latin-1', sep=';')
df_qe[(df_qe['Cargo']=='Deputado Federal') & (df_qe['UF']=='SP')]['Valor do quociente eleitoral']
#df_qe[df_qe['']]

52    333937
Name: Valor do quociente eleitoral, dtype: int64

In [44]:
df_coligacoes = pd.read_csv('dados/candidatos.csv', encoding='latin-1', sep=';')
df_coligacoes = df_coligacoes[['Coligação', 'Federação', 'Sigla partido']].drop_duplicates()
df_coligacoes = df_coligacoes.rename(columns={'Sigla partido':'Partido'})
df_coligacoes['Federação'] = df_coligacoes['Federação'].str.strip()
df_coligacoes.columns = ['coligacao', 'federacao', 'sg_partido']

In [47]:
# Supondo que você já tenha o dataframe df_coligacoes
df_coligacoes.loc[df_coligacoes['federacao'] == '#NULO#', 'federacao'] = df_coligacoes.loc[df_coligacoes['federacao'] == '#NULO#', 'sg_partido']

In [48]:
df_votos = df_votos.join(df_coligacoes.set_index('sg_partido'), on='sg_partido', how='left')

## Valor do Quociente Eleitoral

In [155]:
qe = df_qe[(df_qe['Cargo']=='Deputado Federal') & (df_qe['UF']=='SP')]['Valor do quociente eleitoral'].values[0]
print("O custo de cada cadeira para esse estado é de {} votos".format(int(qe)))

O custo de cada cadeira para esse estado é de 333,937 votos


# Valor da Clausula de Barreira

In [148]:
cb = 0.1*qe
print(" O valor da clausula de barriera é {} votos".format(int(cb)))

 O valor da clausula de barriera é 33393 votos


In [95]:
df_vp = pd.read_csv('dados/votacao_partido.csv', encoding='latin-1', sep=';')
df_vp = df_vp[['Composição da federação', 'Sigla partido', 'Quantidade de votos válidos']]
df_vp.columns = ['federacao', 'sg_partido', 'qt_votos_validos_fede']
df_vp.loc[df_vp['federacao'] == '#NULO#', 'federacao'] = df_vp.loc[df_vp['federacao'] == '#NULO#', 'sg_partido']
df_vp = df_vp.groupby(['federacao']).agg({'qt_votos_validos_fede':'sum'})
df_vp['pc_votos_validos_fede'] = df_vp['qt_votos_validos_fede']/df_vp['qt_votos_validos_fede'].sum()
df_vp['cadeiras'] = df_vp['qt_votos_validos_fede']//qe
df_vp.sort_values('qt_votos_validos_fede', ascending=False)


,qt_votos_validos_fede,pc_votos_validos_fede,cadeiras
federacao,,,
PL,5343667,0.228600,16
PC do B / PT / PV,3132607,0.134012,9
PSOL / REDE,2288861,0.097917,6
UNIÃO,1811462,0.077494,5
MDB,1613626,0.069030,4
REPUBLICANOS,1580891,0.067630,4
CIDADANIA / PSDB,1500112,0.064174,4
PP,1174646,0.050251,3
PSD,1055965,0.045174,3


In [97]:
cad_rest = 70-df_vp['cadeiras'].sum()

10

# Distribuição das vagas que sobraram

Dividir a votação de cada partido pelo número de lugares por ele obtido + 1. 
Ao partido que alcançar a maior média, atribui-se a primeira vaga, desde que tenha candidato com votação mínima 

In [122]:
df_vp['media'] = df_vp['qt_votos_validos_fede']/(df_vp['cadeiras']+1)
df_vp.sort_values('media', ascending=False)


,qt_votos_validos_fede,pc_votos_validos_fede,cadeiras,media
federacao,,,,
PSOL / REDE,2288861,0.097917,6,326980.142857
MDB,1613626,0.069030,4,322725.200000
REPUBLICANOS,1580891,0.067630,4,316178.200000
PL,5343667,0.228600,16,314333.352941
PC do B / PT / PV,3132607,0.134012,9,313260.700000
UNIÃO,1811462,0.077494,5,301910.333333
CIDADANIA / PSDB,1500112,0.064174,4,300022.400000
PODE,892443,0.038178,2,297481.000000
PP,1174646,0.050251,3,293661.500000


In [154]:
dict(zip(list(df_vp[df_vp['cadeiras']>0].federacao.values),list(df_vp[df_vp['cadeiras']>0].cadeiras.values)))

{'CIDADANIA / PSDB': 4,
 'MDB': 4,
 'NOVO': 1,
 'PC do B / PT / PV': 9,
 'PL': 16,
 'PODE': 2,
 'PP': 3,
 'PSB': 2,
 'PSD': 3,
 'PSOL / REDE': 6,
 'REPUBLICANOS': 4,
 'SOLIDARIEDADE': 1,
 'UNIÃO': 5}

A federação PSOL/REDE receberia a vaga, devemos checar se o candidato recebeu mais de 20% do QE

In [127]:
cb = 0.2*qe
print(" O valor da clausula de barriera é {:.2f} votos".format(cb))

 O valor da clausula de barriera é 66787.40 votos


In [129]:
df_votos[df_votos['ds_composicao_coligacao']=='REDE / PSOL'].sort_values('qt_votos_nom_validos', ascending=False).reset_index().head(7)

,index,nm_candidato,sg_partido,ds_composicao_coligacao,ds_sit_totalizacao,pc_votos_validos,qt_votos_nom_validos,qt_votos_concorrentes,coligacao,federacao
0,0,GUILHERME CASTRO BOULOS,PSOL,REDE / PSOL,Eleito,"0,0422",1001472,23750391,FEDERAÇÃO,Federação PSOL REDE
1,8,ERIKA SANTOS SILVA,PSOL,REDE / PSOL,Eleito,"0,0108",256903,23750391,FEDERAÇÃO,Federação PSOL REDE
2,11,MARIA OSMARINA MARINA DA SILVA VAZ DE LIMA,REDE,REDE / PSOL,Eleito,"0,01",237526,23750391,FEDERAÇÃO,Federação PSOL REDE
3,15,SÂMIA DE SOUZA BOMFIM,PSOL,REDE / PSOL,Eleito,"0,0095",226187,23750391,FEDERAÇÃO,Federação PSOL REDE
4,32,SÔNIA BONE DE SOUSA SILVA SANTOS,PSOL,REDE / PSOL,Eleito,"0,0066",156966,23750391,FEDERAÇÃO,Federação PSOL REDE
5,48,LUIZA ERUNDINA DE SOUSA,PSOL,REDE / PSOL,Eleito,"0,0048",113983,23750391,FEDERAÇÃO,Federação PSOL REDE
6,113,LUCIENE CAVALCANTE DA SILVA,PSOL,REDE / PSOL,Suplente,"0,0021",49131,23750391,FEDERAÇÃO,Federação PSOL REDE


Como a candidata LUCIENE CAVALCANTE DA SILVA, recebeu uma votação abaixo da clausula de barreira não recebera a vaga e a proxima federação recebera a vaga.

In [128]:
df_votos[df_votos['ds_composicao_coligacao']=='MDB'].sort_values('qt_votos_nom_validos', ascending=False).head(5)

,index,nm_candidato,sg_partido,ds_composicao_coligacao,ds_sit_totalizacao,pc_votos_validos,qt_votos_nom_validos,qt_votos_concorrentes,coligacao,federacao
0,9,MARIO PALUMBO JUNIOR,MDB,MDB,Eleito,"0,0107",254898,23750391,PARTIDO ISOLADO,MDB
1,12,LUIZ FELIPE BALEIA TENUTO ROSSI,MDB,MDB,Eleito,"0,01",236463,23750391,PARTIDO ISOLADO,MDB
2,13,FÁBIO EDUARDO DE OLIVEIRA TERUEL,MDB,MDB,Eleito,"0,0099",235165,23750391,PARTIDO ISOLADO,MDB
3,47,ALBERTO PEREIRA MOURÃO,MDB,MDB,Eleito,"0,0048",114234,23750391,PARTIDO ISOLADO,MDB
4,61,SIMONE APARECIDA CURRALADAS DOS SANTOS,MDB,MDB,Eleito,"0,0041",97730,23750391,PARTIDO ISOLADO,MDB


In [ ]:
Como a candidata do MDB compriu com a clausula de barreira recebera a vaga.